
# CityBikes


### Send a request to CityBikes for the city of your choice. 

In [27]:
import requests
import json
import pandas as pd

## API request to CityBikes

In [28]:
base_url = r'http://api.citybik.es/v2/networks/'
response = requests.get(base_url)

In [29]:
data = response.json()

In [30]:
#saving json file

with open('citybike_data.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)

In [31]:
#open json file

with open('data/citybike_data.json', 'r') as json_file:
    data = json.load(json_file)

Initial data parsing done only to be lead to another link in order to request the actual data about the bikes stations that I wanted.

In [36]:
data.keys()

dict_keys(['networks'])

In [37]:
data['networks'][0]['location'].keys()

dict_keys(['city', 'country', 'latitude', 'longitude'])

In [38]:
data['networks'][0]['location']['country']

'RU'

In [39]:
data

{'networks': [{'company': ['ЗАО «СитиБайк»'],
   'href': '/v2/networks/velobike-moscow',
   'id': 'velobike-moscow',
   'location': {'city': 'Moscow',
    'country': 'RU',
    'latitude': 55.75,
    'longitude': 37.616667},
   'name': 'Velobike'},
  {'company': ['Urban Infrastructure Partner'],
   'href': '/v2/networks/baerum-bysykkel',
   'id': 'baerum-bysykkel',
   'location': {'city': 'Bærum',
    'country': 'NO',
    'latitude': 59.89455,
    'longitude': 10.546343},
   'name': 'Bysykkel'},
  {'company': ['Comunicare S.r.l.'],
   'href': '/v2/networks/bicincitta-siena',
   'id': 'bicincitta-siena',
   'location': {'city': 'Siena',
    'country': 'IT',
    'latitude': 43.3186,
    'longitude': 11.3306},
   'name': 'Bicincittà',
   'source': 'https://www.bicincitta.com/frmLeStazioni.aspx?ID=202'},
  {'company': ['Cyclopolis Systems'],
   'href': '/v2/networks/cyclopolis-maroussi',
   'id': 'cyclopolis-maroussi',
   'location': {'city': 'Maroussi',
    'country': 'GR',
    'latitude':

In [9]:
#Iterating through 'data' to find information related to Toronto, ON

data_canada = []

for network in data['networks']:
    if network['location']['country'] == 'CA' and network['location']['city']== 'Toronto, ON':
        data_canada.append(network)

print(data_canada)

[{'company': ['Motivate International, Inc.', 'PBSC Urban Solutions'], 'gbfs_href': 'https://tor.publicbikesystem.net/ube/gbfs/v1/', 'href': '/v2/networks/bixi-toronto', 'id': 'bixi-toronto', 'location': {'city': 'Toronto, ON', 'country': 'CA', 'latitude': 43.653226, 'longitude': -79.3831843}, 'name': 'Bike Share Toronto'}]


Above data provided another link to retrieve data for Toronto bike stations.

In [ ]:
#Second API Request to get Toronto information

station_information = requests.get('https://toronto.publicbikesystem.net/customer/ube/gbfs/v1/en/station_information').json()
station_status = requests.get('https://toronto.publicbikesystem.net/customer/ube/gbfs/v1/en/station_status').json()
system_information = requests.get('https://toronto.publicbikesystem.net/customer/ube/gbfs/v1/en/system_information').json()

In [ ]:
#saving files

with open('data/system_information.json', 'w') as json_file:
        json.dump(system_information, json_file, indent=4)

In [40]:
#open json file

with open('data/station_information.json', 'r') as json_file:
    station_information = json.load(json_file)

In [41]:
#open json file

with open('data/station_status.json', 'r') as json_file:
    station_status = json.load(json_file)

### Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [43]:
#sifting through the data in order to understand how it is laid out.

station_information['data']['stations'][0]

{'station_id': '7000',
 'name': 'Fort York  Blvd / Capreol Ct',
 'physical_configuration': 'REGULAR',
 'lat': 43.639832,
 'lon': -79.395954,
 'altitude': 0.0,
 'address': 'Fort York  Blvd / Capreol Ct',
 'capacity': 35,
 'is_charging_station': False,
 'rental_methods': ['KEY', 'TRANSITCARD', 'CREDITCARD', 'PHONE'],
 'groups': [],
 'obcn': '647-643-9607',
 'nearby_distance': 500.0,
 '_ride_code_support': True,
 'rental_uris': {}}

In [45]:
station_status['data']['stations'][2]

{'station_id': '7002',
 'num_bikes_available': 9,
 'num_bikes_available_types': {'mechanical': 8, 'ebike': 1},
 'num_bikes_disabled': 0,
 'num_docks_available': 10,
 'num_docks_disabled': 0,
 'last_reported': 1692901353,
 'is_charging_station': False,
 'status': 'IN_SERVICE',
 'is_installed': 1,
 'is_renting': 1,
 'is_returning': 1,
 'traffic': None}

In [44]:
#grabbing the required information from the station information data

bike_station_data = []

for station in station_information['data']['stations']:
    station_id= station['station_id']
    station_name = station['name']
    station_lat = station['lat']
    station_lon = station['lon']
    bike_station_data.append((station_id, station_name, station_lat, station_lon))

print(bike_station_data)

[('7000', 'Fort York  Blvd / Capreol Ct', 43.639832, -79.395954), ('7001', 'Wellesley Station Green P', 43.66496415990742, -79.38355031526893), ('7002', 'St. George St / Bloor St W', 43.667333, -79.399429), ('7003', 'Madison Ave / Bloor St W', 43.667158, -79.402761), ('7004', 'University Ave / Elm St', 43.656518, -79.389099), ('7005', 'King St W / York St', 43.6480008, -79.383177), ('7006', 'Bay St / College St (East Side)', 43.660439, -79.385525), ('7007', 'College St / Huron St', 43.658148, -79.398167), ('7008', "Wellesley St W / Queen's Park Cres", 43.663376, -79.392125), ('7009', 'King St E / Jarvis St', 43.650325, -79.372287), ('7010', 'King St W / Spadina Ave', 43.645323, -79.395003), ('7012', 'Elizabeth St / Edward St (Bus Terminal)', 43.656026, -79.385327), ('7014', 'Sherbourne St / Carlton St (Allan Gardens)', 43.663102, -79.373181), ('7015', 'King St W / Bay St (West Side)', 43.64852, -79.380576), ('7016', 'Bay St / Queens Quay W (Ferry Terminal)', 43.640978, -79.376785), ('7

Note: The bike numbers used for this project are a snapshop from when the data was acquired.

In [47]:
#collecting number of bikes from the station status data

bike_numbers = []

for bike in station_status['data']['stations']:
    station_id = bike['station_id']
    num_bikes = bike['num_bikes_available']
    bike_numbers.append((station_id, num_bikes))

print(bike_numbers)

[('7000', 6), ('7001', 11), ('7002', 9), ('7003', 2), ('7004', 6), ('7005', 17), ('7006', 2), ('7007', 8), ('7008', 15), ('7009', 16), ('7010', 8), ('7012', 11), ('7014', 5), ('7015', 31), ('7016', 32), ('7018', 9), ('7020', 21), ('7021', 29), ('7022', 11), ('7023', 3), ('7024', 16), ('7025', 1), ('7026', 2), ('7027', 13), ('7028', 16), ('7029', 12), ('7030', 24), ('7031', 7), ('7032', 5), ('7033', 25), ('7034', 4), ('7035', 16), ('7036', 12), ('7037', 17), ('7038', 27), ('7039', 5), ('7040', 1), ('7041', 22), ('7042', 12), ('7043', 4), ('7044', 11), ('7045', 12), ('7046', 10), ('7047', 24), ('7048', 31), ('7049', 11), ('7050', 8), ('7052', 20), ('7053', 25), ('7054', 4), ('7055', 3), ('7056', 4), ('7057', 22), ('7058', 19), ('7059', 6), ('7061', 8), ('7062', 10), ('7063', 15), ('7064', 8), ('7065', 5), ('7066', 15), ('7067', 22), ('7068', 3), ('7069', 7), ('7070', 7), ('7071', 18), ('7072', 5), ('7073', 5), ('7074', 12), ('7075', 20), ('7076', 23), ('7077', 9), ('7078', 13), ('7079', 

### Put your parsed results into a DataFrame.

In [19]:
station_df=pd.DataFrame(bike_station_data)
numbers_df=pd.DataFrame(bike_numbers)

In [48]:
#fixing column headers

station_df.columns = ['station_id', 'name', 'lat', 'lon' ]
numbers_df.columns = ['station_id', 'num_of_bikes']

In [49]:
numbers_df.head(1)

,station_id,num_of_bikes
0,7000,6


In [50]:
station_df.head(1)

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954


In [51]:
#merging above dataframes into one

bike_df = pd.merge(station_df, numbers_df, on='station_id')

In [52]:
bike_df.head(1)

,station_id,name,lat,lon,num_of_bikes
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,6


In [54]:
#saving dataframe to csv

bike_df.to_csv(r'data\bike_df.csv', index=False)